In [16]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

In [17]:
path_, filename_, category_, article_or_summary_,content_ = [],[],[],[],[]
for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        path = os.path.join(dirname, filename).replace("\\","/")
        f = open(os.path.join(dirname, filename),"r")
        try:
            content_.append(str(f.read()))
            path_.append(path)
            filename_.append(filename)
            category_.append(path.split("/")[-2])
            article_or_summary_.append(path.split("/")[-3])
        except:
            print("ERROR ABRIENDO EL FICHERO")

df = pd.DataFrame({"path":path_, "filename":filename_, "category":category_, "article_or_summary":article_or_summary_,"content":content_}, columns=["path", "filename", "category", "article_or_summary","content"])
df

df = df.loc[df['article_or_summary']=='Summaries']

In [18]:
df.head()

,path,filename,category,article_or_summary,content
2224,data/Summaries/business/001.txt,001.txt,business,Summaries,TimeWarner said fourth quarter sales rose 2% t...
2225,data/Summaries/business/002.txt,002.txt,business,Summaries,The dollar has hit its highest level against t...
2226,data/Summaries/business/003.txt,003.txt,business,Summaries,Yukos' owner Menatep Group says it will ask Ro...
2227,data/Summaries/business/004.txt,004.txt,business,Summaries,"Rod Eddington, BA's chief executive, said the ..."
2228,data/Summaries/business/005.txt,005.txt,business,Summaries,Pernod has reduced the debt it took on to fund...


In [19]:
df = df[['category','content']]

In [20]:
df.head()

,category,content
2224,business,TimeWarner said fourth quarter sales rose 2% t...
2225,business,The dollar has hit its highest level against t...
2226,business,Yukos' owner Menatep Group says it will ask Ro...
2227,business,"Rod Eddington, BA's chief executive, said the ..."
2228,business,Pernod has reduced the debt it took on to fund...


In [21]:
# Dividir el DataFrame en características (X) y etiquetas (y)
X = df['content']
y = df['category']

# Crear un vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Transformar los textos en vectores TF-IDF
X_vectorized = vectorizer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Clasificador Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predecir las etiquetas de los datos de prueba
y_pred = classifier.predict(X_test)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.97      0.92      0.95       115
entertainment       0.98      0.79      0.88        72
     politics       0.88      0.97      0.93        76
        sport       0.90      0.99      0.94       102
         tech       0.94      0.96      0.95        80

     accuracy                           0.93       445
    macro avg       0.94      0.93      0.93       445
 weighted avg       0.94      0.93      0.93       445



### PRUEBA CON LOS RESÚMENES GENERADOR POR HUGGINGFACE

In [22]:
HF_Resumenes = pd.read_csv("ResumenesGeneradosHF.csv")
HF_Resumenes.head()

,filename,summary_HF,category
0,001.txt,Time Warner profits up 76% to $1.13bn for the ...,business
1,002.txt,Dollar hits highest level against the euro in ...,business
2,003.txt,Yukos' owner Menatep Group says it will ask Ro...,business
3,004.txt,British Airways blames high fuel prices for a ...,business
4,005.txt,Allied Domecq shares in London rose 4% by 1200...,business


In [23]:
x_generados_HF = HF_Resumenes['summary_HF']
X__HF_vectorized = vectorizer.transform(x_generados_HF)
y_HF = HF_Resumenes['category']
y_pred_HF = classifier.predict(X__HF_vectorized)
print(classification_report(y_HF, y_pred_HF))

               precision    recall  f1-score   support

     business       1.00      0.93      0.97        15
entertainment       1.00      0.93      0.97        15
     politics       0.83      1.00      0.91        15
        sport       1.00      1.00      1.00        15
         tech       1.00      0.88      0.93         8

     accuracy                           0.96        68
    macro avg       0.97      0.95      0.95        68
 weighted avg       0.96      0.96      0.96        68

